In [1]:
from transformers import BertTokenizer, AutoModelForSequenceClassification
from transformers_interpret import SequenceClassificationExplainer
from readers import lenta_reader, ria_reader
from tqdm.notebook import tqdm
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
tokenizer_path = '/home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/'
model_path = '/home/aobuhtijarov/models/discriminator_on_clusters_from_rubert/''
lenta_path = '/home/aobuhtijarov/datasets/lenta/lenta-ru-news.test.csv'
ria_path = '/home/aobuhtijarov/datasets/ria/ria.shuffled.test.json'

In [3]:
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=False, do_basic_tokenize=False)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, output_attentions=True)

In [4]:
import json

def reader(path):
    with open(path, 'r') as f:
        for line in f:
            yield json.loads(line.strip())
            
            
records = [r for r in tqdm(reader('../../datasets/full_lenta_ria.test.jsonl'))]

0it [00:00, ?it/s]

In [5]:
lenta_records = [
    {'text': r['lenta_text'], 'title': r['lenta_title'], 'agency': 'lenta.ru', 'date': r['lenta_date']} 
    for r in records
]

ria_records = [
    {'text': r['ria_text'], 'title': r['ria_title'], 'agency': 'РИА Новости', 'date': r['lenta_date']} 
    for r in records
]

In [6]:
agency_list = ["РИА Новости", "lenta.ru"]
agency_to_special_token_id = {a: tokenizer.vocab[f'[unused{i+1}]'] for i, a in enumerate(agency_list)}
agency_to_discr_target = {a: i for i, a in enumerate(sorted(agency_list))}
discr_target_to_agency = {v: k for k, v in agency_to_discr_target.items()}

cname_to_agency = { 
    f'LABEL_{i}': discr_target_to_agency[i] for i in range(len(agency_list))
}

In [7]:
# lenta_records = [r for r in tqdm(lenta_reader(lenta_path))][:20000]
# ria_records = [r for r in tqdm(ria_reader(ria_path))][:20000]

In [8]:
# n = random.randint(0, 43000)
# lenta_records[n]['title'], ria_records[n]['title']

## Predictions

In [9]:
# model.cuda();

In [10]:
cls_explainer = SequenceClassificationExplainer(model, tokenizer)

### Lenta

In [11]:
lenta_results = []
lenta_pred_cnt = 0

for r in tqdm(lenta_records, total=len(lenta_records)):
    if len(r['title']) > 150:
        print(r)
        continue
    
    val = cls_explainer(r['title'].lower())
    
    if cname_to_agency[cls_explainer.predicted_class_name] == 'lenta.ru':
        lenta_pred_cnt += 1
        lenta_results.extend(val)


  0%|          | 0/2000 [00:00<?, ?it/s]

{'text': 'экономика', 'title': '"газпром" и турецкая botas подписали дополнения к долгосрочным контрактам на поставки газа, согласно которым объем экспорта топлива в турцию увеличится как минимум на 2 миллиарда кубометров. в 2011 году поставки газа в страну составят 25,8 миллиарда кубометров, что, по словам главы монополии алексея миллера, является абсолютным рекордом в рамках российско-турецкого сотрудничества. об этом сообщает "интерфакс". россия поставляет турции газ по трем контрактам: один по "голубому потоку" и два по западному маршруту транзитом через украину. действие одного из них, подписанного в 1986 году сроком на 25 лет, заканчивается в этом году. на переговорах 28 декабря botas и "газпром" подписали дополнения к двум другим контрактам: 25-летнего соглашения от 1997 года на поставки по "голубому потоку" и 23-летнего договора от 1998 года на поставки через украину. ранее botas отказалась продлевать соглашение 1986 года, пытаясь добиться от "газпрома" 20-процентной скидки. со

In [12]:
print('Lenta accuracy:', round(lenta_pred_cnt / len(lenta_records), 2))

Lenta accuracy: 0.89


### RIA

In [13]:
ria_results = []
ria_pred_cnt = 0

for r in tqdm(ria_records, total=len(ria_records)):
    if len(r['title']) > 150:
        print(r)
        continue

    val = cls_explainer(r['title'].lower())
    if cname_to_agency[cls_explainer.predicted_class_name] == 'РИА Новости':
        ria_pred_cnt += 1
        ria_results.extend(val)


  0%|          | 0/2000 [00:00<?, ?it/s]

In [14]:
print('RIA accuracy:', round(ria_pred_cnt / len(ria_records), 2))

RIA accuracy: 0.85


### General discriminator analysis

In [15]:
def extract_top_n_words(res, percentile=85, n=20):
    p = np.percentile([x[1] for x in res], percentile)
    top_tokens = [x[0] for x in res if x[1] >= p]
    return Counter(top_tokens).most_common(n)

In [25]:
lenta_results = [(a, b) for a, b in lenta_results if a not in ['[CLS]', '[SEP]']]
ria_results = [(a, b) for a, b in ria_results if a not in ['[CLS]', '[SEP]']]

`
Positive attribution numbers indicate a word contributes positively towards the predicted class, while negative numbers indicate a word contributes negatively towards the predicted class
`

#### Lenta

Positive

In [26]:
extract_top_n_words(lenta_results)

[('в', 137),
 ('##-', 43),
 ('##а', 37),
 ('за', 36),
 ('о', 34),
 ('##и', 34),
 ('на', 33),
 ('##е', 30),
 ('с', 17),
 ('об', 13),
 ('##о', 12),
 ('##у', 12),
 ('и', 12),
 ('умер', 11),
 ('##ев', 11),
 ('по', 10),
 ('потребовали', 10),
 ('предъявили', 9),
 ('заподозрили', 9),
 ('##н', 9)]

Negative

In [27]:
extract_top_n_words([(x[0], -x[1]) for x in lenta_results])

[('"', 192),
 ('##"', 126),
 ('в', 109),
 ('на', 94),
 ('##ии', 47),
 ('##а', 26),
 ('за', 22),
 ('от', 20),
 ('из', 18),
 ('##ы', 18),
 ('по', 18),
 ('о', 17),
 ('росс', 16),
 ('и', 15),
 ('##-', 14),
 ('##е', 14),
 ('##и', 13),
 ('с', 12),
 ('##н', 12),
 ('москв', 12)]

#### RIA

Positive

In [28]:
extract_top_n_words(ria_results)

[(',', 279),
 ('в', 220),
 ('##:', 87),
 ('-', 74),
 ('"', 39),
 ('$', 37),
 ('млрд', 37),
 ('млн', 27),
 ('##%', 26),
 ('на', 26),
 ('может', 25),
 ('##е', 25),
 ('##ф', 23),
 ('руб', 23),
 ('г', 23),
 ('о', 21),
 ('по', 21),
 ('из', 18),
 ('##ии', 17),
 ('экс', 16)]

Negative

In [29]:
extract_top_n_words([(x[0], -x[1]) for x in ria_results])

[('в', 112),
 ('##-', 42),
 ('на', 39),
 ('о', 23),
 ('с', 22),
 (',', 22),
 ('москв', 21),
 ('р', 20),
 ('и', 20),
 ('по', 18),
 ('человек', 18),
 ('сш', 16),
 ('##ии', 16),
 ('##ф', 14),
 ('"', 14),
 ('из', 13),
 ('##"', 13),
 ('##н', 13),
 ('под', 12),
 ('против', 12)]

### Visualization on concrete examples

In [15]:
for target, sent in (
        ('Lenta', 'в столичной квартире нашли тела пенсионерки и ее сына-инвалида'),
        ('Lenta', 'американцы начали подавать иски против китая из-за коронавируса'),
        ('Lenta', 'стал известен источник полученных тимошенко миллионов долларов'),
        ('Lenta', 'названо преимущество полноценного сна перед конфетами'),
        ('TASS', 'коллекторы считают, что 40% сотрудников компаний могут просрочить платежи по кредитам'),
        ('TASS', 'более 40 военнослужащих рф, задействованных в борьбе с пандемией, вернулись из италии'),
        ('RT', 'климкин заявил об отсутствии у россии «права» праздновать день победы'),
        ('RT', 'в минобороны прокомментировали возвращение специалистов из италии'),
    
    ):
    cls_explainer(sent.lower());
    print(' ' * 72 + f'True: {target}; Pred: {cname_to_agency[cls_explainer.predicted_class_name]}')
    cls_explainer.visualize('bert-discr-vis.html')

                                                                        True: Lenta; Pred: Lenta


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,LABEL_1 (0.89),LABEL_1,1.22,[CLS] в столичной квартире нашли тела пенсионер ##ки и ее сына ##- ##ин ##вали ##да [SEP]


                                                                        True: Lenta; Pred: Lenta


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,LABEL_1 (1.00),LABEL_1,2.91,[CLS] американцы начали подавать иски против кита ##я из ##- ##за корона ##вир ##уса [SEP]


                                                                        True: Lenta; Pred: Lenta


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,LABEL_2 (1.00),LABEL_2,1.63,"[CLS] более 40 военнослужащих р ##ф , задействованных в борьбе с панд ##емией , вернулись из итал ##ии [SEP]"


                                                                        True: RT; Pred: RT


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,LABEL_0 (0.97),LABEL_0,1.23,[CLS] клим ##кин заявил об отсутствии у росс ##ии « ##прав ##а ##» праздновать день победы [SEP]


                                                                        True: RT; Pred: RT


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,LABEL_0 (1.00),LABEL_0,2.31,[CLS] в минобороны прокомментировали возвращение специалистов из итал ##ии [SEP]


In [12]:
cls_explainer(sent);

Positive attribution numbers indicate a word contributes positively towards the predicted class, while negative numbers indicate a word contributes negatively towards the predicted class. 

In [13]:
cname_to_agency[cls_explainer.predicted_class_name]

'Lenta'

In [15]:

cls_explainer.visualize('bert-discr-vis.html');

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,LABEL_1 (0.89),LABEL_1,1.22,[CLS] в столичной квартире нашли тела пенсионер ##ки и ее сына ##- ##ин ##вали ##да [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,LABEL_1 (0.89),LABEL_1,1.22,[CLS] в столичной квартире нашли тела пенсионер ##ки и ее сына ##- ##ин ##вали ##да [SEP]


### Layers attention

In [110]:
inputs = tokenizer.encode_plus(sent, return_tensors='pt', add_special_tokens=True)
input_ids = inputs['input_ids'].cuda()
token_type_ids = inputs['token_type_ids'].cuda()

In [111]:
out = model(input_ids, token_type_ids=token_type_ids)


In [112]:
attention = out[-1]

In [113]:
# sentence_b_start = token_type_ids[0].tolist().index(1)
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list) 

In [114]:
model_view(attention, tokens)

<IPython.core.display.Javascript object>

In [115]:
head_view(attention, tokens)

<IPython.core.display.Javascript object>